# Notebook for building a model for the Sendfrid sign up dataset
***
**Jake Mitchell Scott Schubert**

Initially using ______

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
#from xgboost import XGB
from sklearn.model_selection import train_test_split

In [7]:
train_path = 'data/signup_train_data.csv'

# Load the data into a DataFrame 
df = pd.read_csv(train_path, low_memory=False)
# Split into X and Y
dfLabels = df.pop("label")
#get train and valid sets
X_train, X_valid, y_train, y_valid = train_test_split(baseData, baseLabels, train_size=0.5, random_state=1000)
print(X_train.head())

NameError: name 'baseData' is not defined